In [1]:
# %load ../import_data.py
from collections import Counter
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 20)

import xgboost as xgb

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from statistic import Statistic
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction

# for autoreload modules
%load_ext autoreload
%autoreload 2

# path_data = '/mnt/files/workdata/work/merchants/merchant_32_ky9_2020-05-12_white_visa/04_experiments/'
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

In [2]:
import csv
path_merch = '/mnt/files/workdata/work/merchants/merchant_35_ky_2021-03-12/'
# /mnt/files/workdata/work/merchants/merchant_35_ky_2021-03-12/01_raw_data

In [3]:
path_df = path_merch + '01_raw_data/20210313-canis-vulgaris-for-ml-operations.csv'
path_df_bad =  path_merch + '01_raw_data/20210313-canis-vulgaris-for-ml-fraud.csv'

In [4]:
df_100 = pd.read_csv(path_df, sep='\t' ,dtype=str, nrows=99, keep_default_na=False)

In [5]:
str(list(df_100))

"['id', 'order_id', 'type', 'status', 'amount', 'currency', 'terminal_id', 'mcc', 'project_id', 'created', 'pan', 'expiration_year', 'expiration_month', 'country', 'zip', 'phone', 'ip']"

In [6]:
df_100.head()

,id,order_id,type,status,amount,currency,terminal_id,mcc,project_id,created,pan,expiration_year,expiration_month,country,zip,phone,ip
0,183115710,58963194491047906,authorize,failure,25.32,USD,568a5ee9d6625d4c61738b836e8e66356dc61ae6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:09:39,514841****5179,22,5,USA,13215,3155602060,72.237.4.249
1,183115782,58963204904455627,authorize,success,141.68,USD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:10:20,448233****3385,22,4,USA,06248,8608696059,73.149.7.42
2,183115785,58963204904455627,charge,success,141.68,USD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:10:22,448233****3385,22,4,USA,06248,8608696059,73.149.7.42
3,183115905,58963219744750136,authorize,success,26.83,USD,7e056fed4960c5b2f58c35a1e4873dc5e687d4c6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:11:18,438787****8595,24,11,USA,11590,5163857019,174.197.204.100
4,183115916,58963219744750136,charge,success,26.83,USD,7e056fed4960c5b2f58c35a1e4873dc5e687d4c6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:11:20,438787****8595,24,11,USA,11590,5163857019,174.197.204.100


In [7]:
df =  pd.read_csv(path_df, sep='\t' ,dtype=str, keep_default_na=False)

In [8]:
df.shape[0]

468821

In [9]:
df.tail()

,id,order_id,type,status,amount,currency,terminal_id,mcc,project_id,created,pan,expiration_year,expiration_month,country,zip,phone,ip
468816,190689545,59819743760722526,authorize,success,103.58,CAD,4284af4c1f34eb9d02ac9116bb4e554ddf728f64,7311,ad3295ea1672533f0892043933e3d1b44a8aa359,2021-03-13 14:34:35,522303****5829,25,5,CAN,M6S-5B3,6478825506,72.138.207.35
468817,190689557,59819743760722526,charge,success,103.58,CAD,4284af4c1f34eb9d02ac9116bb4e554ddf728f64,7311,ad3295ea1672533f0892043933e3d1b44a8aa359,2021-03-13 14:34:37,522303****5829,25,5,CAN,M6S-5B3,6478825506,72.138.207.35
468818,190689631,59819747759984116,authorize,failure,108.98,USD,4284af4c1f34eb9d02ac9116bb4e554ddf728f64,7311,ad3295ea1672533f0892043933e3d1b44a8aa359,2021-03-13 14:34:50,517805****5516,22,7,USA,91306,8162889435,136.34.79.4
468819,190689739,59819759416841735,authorize,success,759.41,AUD,4284af4c1f34eb9d02ac9116bb4e554ddf728f64,7311,ad3295ea1672533f0892043933e3d1b44a8aa359,2021-03-13 14:35:36,543568****0083,25,11,AUS,6430,417982032,180.150.81.214
468820,190689744,59819759416841735,charge,success,759.41,AUD,4284af4c1f34eb9d02ac9116bb4e554ddf728f64,7311,ad3295ea1672533f0892043933e3d1b44a8aa359,2021-03-13 14:35:38,543568****0083,25,11,AUS,6430,417982032,180.150.81.214


In [10]:
df_bad =  pd.read_csv(path_df_bad, sep='\t' ,dtype=str, keep_default_na=False)

In [11]:
df_bad.head(1)

,id,status,date,reason,fraud_type,report_date,created,terminal_id,mcc,project_id,created.1,pan,expiration_year,expiration_month,country,zip,phone,ip
0,58989100919705840,chargedback,2021-02-07,fraudreport,6,2021-02-09,2021-02-09 16:02:36,568a5ee9d6625d4c61738b836e8e66356dc61ae6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-04 01:16:15,481582****9076,24,10,USA,02494,5104488968,172.58.35.224


In [12]:
df_bad.status.value_counts()

chargedback    913
charged        275
refunded        15
Name: status, dtype: int64

In [13]:
df_bad.shape[0]

1203

In [17]:
len(df_bad.id.unique())

1203

In [18]:
str(list(df))

"['id', 'order_id', 'type', 'status', 'amount', 'currency', 'terminal_id', 'mcc', 'project_id', 'created', 'pan', 'expiration_year', 'expiration_month', 'country', 'zip', 'phone', 'ip']"

In [19]:
str(list(df_bad))

"['id', 'status', 'date', 'reason', 'fraud_type', 'report_date', 'created', 'terminal_id', 'mcc', 'project_id', 'created.1', 'pan', 'expiration_year', 'expiration_month', 'country', 'zip', 'phone', 'ip']"

#####  df.created - дата создания ордера
#####  df_bad.reported - дата прихода fraud)report

In [20]:
min(df.created)

'2021-02-02 21:09:39'

In [21]:
max(df.created)

'2021-03-13 14:40:07'

In [23]:
min(df_bad.report_date)

'2021-02-09'

In [24]:
max(df_bad.report_date)

'2021-03-12'

#### Duble id check n

In [25]:
df_duplicated = df[df.order_id.duplicated(keep=False)].copy()

In [26]:
df_duplicated.shape[0]

413624

In [27]:
df_duplicated.sort_values(by=['order_id', 'created'], inplace=True)

In [28]:
df_duplicated.head(6)

,id,order_id,type,status,amount,currency,terminal_id,mcc,project_id,created,pan,expiration_year,expiration_month,country,zip,phone,ip
1,183115782,58963204904455627,authorize,success,141.68,USD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:10:20,448233****3385,22,4,USA,06248,8608696059,73.149.7.42
2,183115785,58963204904455627,charge,success,141.68,USD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:10:22,448233****3385,22,4,USA,06248,8608696059,73.149.7.42
3,183115905,58963219744750136,authorize,success,26.83,USD,7e056fed4960c5b2f58c35a1e4873dc5e687d4c6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:11:18,438787****8595,24,11,USA,11590,5163857019,174.197.204.100
4,183115916,58963219744750136,charge,success,26.83,USD,7e056fed4960c5b2f58c35a1e4873dc5e687d4c6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:11:20,438787****8595,24,11,USA,11590,5163857019,174.197.204.100
6,183116260,58963261615776513,authorize,success,34.77,USD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:01,481582****0912,24,1,USA,91801,7147975264,107.77.229.166
7,183116266,58963261615776513,charge,success,34.77,USD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:03,481582****0912,24,1,USA,91801,7147975264,107.77.229.166


In [29]:
df_duplicated.type.value_counts()

charge        206112
authorize     206111
chargeback      1109
refund           292
Name: type, dtype: int64

In [30]:
df_duplicated.status.value_counts()

success    413618
error           6
Name: status, dtype: int64

In [31]:
df_not_duplicated = df[~df.order_id.duplicated(keep=False)].copy()

In [32]:
df_not_duplicated.sort_values(by=['order_id', 'created'], inplace=True)

In [33]:
df_not_duplicated.head()

,id,order_id,type,status,amount,currency,terminal_id,mcc,project_id,created,pan,expiration_year,expiration_month,country,zip,phone,ip
0,183115710,58963194491047906,authorize,failure,25.32,USD,568a5ee9d6625d4c61738b836e8e66356dc61ae6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:09:39,514841****5179,22,5,USA,13215,3155602060,72.237.4.249
5,183116236,58963258822580831,authorize,failure,103.48,CAD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:13:50,452088****3744,24,11,CAN,M8v 3w9,6472784126,207.107.141.126
12,183116455,58963287116739040,authorize,failure,103.51,CAD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:15:41,452088****3744,21,11,CAN,M8v 3w9,6472784126,207.107.141.126
17,183117108,58963369718922947,authorize,failure,62.12,CAD,568a5ee9d6625d4c61738b836e8e66356dc61ae6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:21:03,408635****3370,24,6,CAN,N4k 6h2,5194774624,99.249.249.113
18,183117315,58963393441347837,authorize,failure,52.00,CAD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:22:36,536088****5564,23,2,CAN,N5v2l2,2266785891,209.171.88.79


In [34]:
df_not_duplicated.status.value_counts()

failure    54365
error        823
success        9
Name: status, dtype: int64

In [35]:
mask = df_not_duplicated.status == 'success'
df_not_duplicated_success = df_not_duplicated[mask]
df_not_duplicated_success.head()

,id,order_id,type,status,amount,currency,terminal_id,mcc,project_id,created,pan,expiration_year,expiration_month,country,zip,phone,ip
233407,184394362,59125651025112720,authorize,success,51.91,USD,ed316d0a28571ee2a0da5ef9c8c8743370270387,7994,3ee0400e7da4aa7e01269c5bd634ab9a7a7f25a2,2021-02-10 05:26:15,486796****8998,25,12,USA,33301,3306103274,107.77.206.102
216537,185359559,59254040816793416,authorize,success,1057.56,USD,a277488212bae45e194e272d40721db81ff7b507,5945,a2126af5a0d9c2bbafd7e34759a6815fa1d5e27d,2021-02-16 00:44:59,519667****7733,24,6,USA,91367,9413036460,47.199.84.28
269626,186435457,59387674580933924,authorize,success,54.78,USD,ed316d0a28571ee2a0da5ef9c8c8743370270387,7994,3ee0400e7da4aa7e01269c5bd634ab9a7a7f25a2,2021-02-22 01:45:07,400022****7351,25,1,USA,97236,8502654682,107.77.237.152
108002,186445623,59389546665885549,authorize,success,25.37,AUD,7e056fed4960c5b2f58c35a1e4873dc5e687d4c6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-22 03:46:58,521729****9941,22,11,AUS,3775,0434484319,198.8.81.199
140548,188225167,59565639729762165,authorize,success,52.00,AUD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-03-02 02:51:28,516361****3940,22,7,AUS,2153,430350851,120.17.28.51


In [36]:
len(df.id.unique())

468821

In [37]:
len(df.order_id.unique())

261308

In [38]:
# Count Double ids 
df.shape[0] - len(df.id.unique())

0

In [39]:
# Count Double order_id
df.shape[0] - len(df.order_id.unique())

207513

In [40]:
df.groupby(['status', 'type']).size().unstack(fill_value=0)

type,authorize,charge,chargeback,refund
status,,,,
error,823,6,0,0
failure,54365,0,0,0
success,206120,206106,1109,292


In [41]:
Counter(df.status)

Counter({'failure': 54365, 'success': 413627, 'error': 829})

# бЫЛО РАНЬШЕ
# Выбираем уникальные записи по order_id:
# Это нужно для того, чтобы можно было проставить дату возврата, для плохих записей у которых нет fruad report
#ky <- filter(ky9, type=='chargeback' | ((type=='charge') & !(id %in% ky9_type_cb$id)))

In [42]:
df.type.value_counts()

authorize     261308
charge        206112
chargeback      1109
refund           292
Name: type, dtype: int64

##### Выбираем ордера type == charge
##### Выбрасываем status == error
##### JOIN с fRAUD B и проставляем статус согласно колонке reported
##### JOIN с (type == 'chargeback') 
##### заменяем значение поля кузщкеув для ордеров у которых не пустое поле reported_chargeback а поле reported пустое
##### проставляем статусы согласно полю reported_chargeback
##### делаем колонку date_only

In [43]:
mask = df.type == 'chargeback'
df_chargeback = df[mask].copy()

In [44]:
mask = df.type == 'charge'
df_charge = df[mask].copy()

In [45]:
df_charge.shape

(206112, 17)

In [46]:
df_charge.status.value_counts()

success    206106
error           6
Name: status, dtype: int64

In [47]:
mask = df_charge.status != 'error'
df_charge = df_charge[mask].copy()

In [48]:
df_charge.status.value_counts()

success    206106
Name: status, dtype: int64

In [49]:
len(df_charge.order_id.unique())

206106

In [50]:
col_names = ['id', 'created']
df_bad_order_id = df_bad[col_names].copy()

In [51]:
df_bad_order_id.columns = ['order_id', 'reported']

In [52]:
df_charge = pd.merge(df_charge, df_bad_order_id, how='left', on=['order_id'])

In [53]:
df_charge['status_old'] = df_charge['status'].copy()

In [54]:
df_charge['status'] =  np.where( pd.notnull(df_charge['reported']), 'true', 'false')

In [55]:
df_charge.status.value_counts()

false    204903
true       1203
Name: status, dtype: int64

In [56]:
df_bad.shape

(1203, 18)

In [57]:
mask = df_chargeback.order_id.isin(df_bad.id)
chargeback_notin_fraud = df_chargeback[~mask]
chargeback_notin_fraud.shape[0]

196

In [58]:
col_names = ['order_id', 'created']
df_chargeback_reported = df_chargeback[col_names]
df_chargeback_reported.columns = ['order_id', 'reported_chargeback']

In [59]:
df_charge = pd.merge(df_charge, df_chargeback_reported, how='left', on=['order_id'])

In [60]:
mask = (pd.isnull(df_charge['reported'])) & (pd.notnull(df_charge['reported_chargeback']))
df_charge['reported'] = np.where( mask , df_charge['reported_chargeback'], df_charge['reported'] )

In [61]:
mask = (pd.notnull(df_charge['reported_chargeback']))
df_charge['status'] = np.where( mask , 'true', df_charge['status'])

In [62]:
df_charge.status.value_counts()

false    204707
true       1399
Name: status, dtype: int64

In [63]:
df_charge['date_cb_only'] = df_charge['reported'].str[:10]
df_charge['date_only'] = df_charge['created'].str[:10]

In [64]:
df_charge['card_masked'] = df_charge.pan.str[:6] + "******" + df_charge.pan.str.slice(10, 14)

In [65]:
df_charge['expire_year'] = "20" + df_charge.expiration_year
df_charge['expire_month'] = np.where(df_charge.expiration_month.str.len() == 1, "0" + df_charge.expiration_month, df_charge.expiration_month)
df_charge['card_expire'] = df_charge.expire_year +  "-" +  df_charge.expire_month

In [66]:
df_charge['id'] = df_charge.card_masked + '_' + df_charge.card_expire

In [67]:
df_charge.groupby(['currency']).size()

currency
ARS      1049
AUD     11806
BRL        29
CAD     20547
USD    172675
dtype: int64

In [68]:
df_charge['amount_usd'] = df_charge.amount.copy()
df_charge.amount_usd = pd.to_numeric(df_charge.amount_usd, errors="coerce")

In [69]:
# USD_EUR = 0.90
USD_CAD = 1.30
# USD_INR = 71.13
# USD_JPY = 108.94

USD_ARS = 90.79
USD_AUD = 1.29
USD_BRL = 5.55

In [70]:
df_charge.loc[df_charge.currency == "CAD",'amount_usd'] = df_charge.loc[df_charge.currency == "CAD",'amount_usd'] / USD_CAD
df_charge.loc[df_charge.currency == "ARS",'amount_usd'] = df_charge.loc[df_charge.currency == "ARS",'amount_usd'] / USD_ARS
df_charge.loc[df_charge.currency == "AUD",'amount_usd'] = df_charge.loc[df_charge.currency == "AUD",'amount_usd'] / USD_AUD
df_charge.loc[df_charge.currency == "BRL",'amount_usd'] = df_charge.loc[df_charge.currency == "BRL",'amount_usd'] / USD_BRL

In [71]:
df_charge.amount_usd.mean()

111.80133487106995

In [72]:
min(df_charge.amount_usd)

0.9675675675675676

In [73]:
max(df_charge.amount_usd)

2014.52

In [74]:
df_charge.amount_usd.median()

54.77

In [75]:
mask = (df_charge.amount_usd > 1850.00)
col_names = ['amount', 'amount_usd', 'currency', 'id', 'order_id','status']
tmp = df_charge[mask]
tmp[col_names]

,amount,amount_usd,currency,id,order_id,status
91818,2004.46,2004.46,USD,549110******9647_2023-06,59077624974419050,false
91908,2014.23,2014.23,USD,545212******4930_2025-07,59077715850975583,false
95314,1910.83,1910.83,USD,521876******1160_2023-03,59210465918117656,false
95569,2013.83,2013.83,USD,512230******4959_2024-07,59232913187686960,false
95585,2014.52,2014.52,USD,552433******7818_2023-06,59235484633928799,false


In [76]:
df_charge.amount_usd = df_charge.amount_usd.round(2).astype(str)

In [78]:
str(list(df_charge))

"['id', 'order_id', 'type', 'status', 'amount', 'currency', 'terminal_id', 'mcc', 'project_id', 'created', 'pan', 'expiration_year', 'expiration_month', 'country', 'zip', 'phone', 'ip', 'reported', 'status_old', 'reported_chargeback', 'date_cb_only', 'date_only', 'card_masked', 'expire_year', 'expire_month', 'card_expire', 'amount_usd']"

In [80]:
drop_columns = ['pan', 'expiration_year', 'expiration_month', 'expire_year', 'expire_month']
df_charge.drop(columns=drop_columns, axis=1, inplace=True)

In [81]:
df_charge.head()

,id,order_id,type,status,amount,currency,terminal_id,mcc,project_id,created,...,phone,ip,reported,status_old,reported_chargeback,date_cb_only,date_only,card_masked,card_expire,amount_usd
0,448233******3385_2022-04,58963204904455627,charge,false,141.68,USD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:10:22,...,8608696059,73.149.7.42,NaN,success,NaN,NaN,2021-02-02,448233******3385,2022-04,141.68
1,438787******8595_2024-11,58963219744750136,charge,false,26.83,USD,7e056fed4960c5b2f58c35a1e4873dc5e687d4c6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:11:20,...,5163857019,174.197.204.100,NaN,success,NaN,NaN,2021-02-02,438787******8595,2024-11,26.83
2,481582******0912_2024-01,58963261615776513,charge,false,34.77,USD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:03,...,7147975264,107.77.229.166,NaN,success,NaN,NaN,2021-02-02,481582******0912,2024-01,34.77
3,510277******8427_2024-08,58963268351715157,charge,false,25.32,USD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:29,...,3155602060,72.237.4.249,NaN,success,NaN,NaN,2021-02-02,510277******8427,2024-08,25.32
4,472409******4012_2021-09,58963268900246649,charge,false,25.34,CAD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:31,...,9059609406,99.254.60.245,NaN,success,NaN,NaN,2021-02-02,472409******4012,2021-09,19.49


In [82]:
df_charge.rename(inplace=True, columns={ 'created': 'date' })

In [83]:
df_charge.head()

,id,order_id,type,status,amount,currency,terminal_id,mcc,project_id,date,...,phone,ip,reported,status_old,reported_chargeback,date_cb_only,date_only,card_masked,card_expire,amount_usd
0,448233******3385_2022-04,58963204904455627,charge,false,141.68,USD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:10:22,...,8608696059,73.149.7.42,NaN,success,NaN,NaN,2021-02-02,448233******3385,2022-04,141.68
1,438787******8595_2024-11,58963219744750136,charge,false,26.83,USD,7e056fed4960c5b2f58c35a1e4873dc5e687d4c6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:11:20,...,5163857019,174.197.204.100,NaN,success,NaN,NaN,2021-02-02,438787******8595,2024-11,26.83
2,481582******0912_2024-01,58963261615776513,charge,false,34.77,USD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:03,...,7147975264,107.77.229.166,NaN,success,NaN,NaN,2021-02-02,481582******0912,2024-01,34.77
3,510277******8427_2024-08,58963268351715157,charge,false,25.32,USD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:29,...,3155602060,72.237.4.249,NaN,success,NaN,NaN,2021-02-02,510277******8427,2024-08,25.32
4,472409******4012_2021-09,58963268900246649,charge,false,25.34,CAD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:31,...,9059609406,99.254.60.245,NaN,success,NaN,NaN,2021-02-02,472409******4012,2021-09,19.49


In [89]:
df_charge.status.value_counts()

false    204707
true       1399
Name: status, dtype: int64

In [90]:
file_to_resolve_path = path_merch + "02_pure_data/ky10_for_resolve.csv"


In [91]:
file_to_resolve_path

'/mnt/files/workdata/work/merchants/merchant_35_ky_2021-03-12/02_pure_data/ky10_for_resolve.csv'

In [92]:
df_charge.to_csv(file_to_resolve_path, index=False, quoting=csv.QUOTE_ALL)

In [109]:
str(list(df_charge))

"['id', 'order_id', 'type', 'status', 'amount', 'currency', 'terminal_id', 'mcc', 'project_id', 'date', 'country', 'zip', 'phone', 'ip', 'reported', 'status_old', 'reported_chargeback', 'date_cb_only', 'date_only', 'card_masked', 'card_expire', 'amount_usd']"

In [108]:
df_charge.head()

,id,order_id,type,status,amount,currency,terminal_id,mcc,project_id,date,country,zip,phone,ip,reported,status_old,reported_chargeback,date_cb_only,date_only,card_masked,card_expire,amount_usd
0,448233******3385_2022-04,58963204904455627,charge,false,141.68,USD,6e47c22917615e7d98fd3bdd6c9a9aa46801d326,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:10:22,USA,06248,8608696059,73.149.7.42,NaN,success,NaN,NaN,2021-02-02,448233******3385,2022-04,141.68
1,438787******8595_2024-11,58963219744750136,charge,false,26.83,USD,7e056fed4960c5b2f58c35a1e4873dc5e687d4c6,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:11:20,USA,11590,5163857019,174.197.204.100,NaN,success,NaN,NaN,2021-02-02,438787******8595,2024-11,26.83
2,481582******0912_2024-01,58963261615776513,charge,false,34.77,USD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:03,USA,91801,7147975264,107.77.229.166,NaN,success,NaN,NaN,2021-02-02,481582******0912,2024-01,34.77
3,510277******8427_2024-08,58963268351715157,charge,false,25.32,USD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:29,USA,13215,3155602060,72.237.4.249,NaN,success,NaN,NaN,2021-02-02,510277******8427,2024-08,25.32
4,472409******4012_2021-09,58963268900246649,charge,false,25.34,CAD,39ab5284a1a698f98c2a8712a97e9b78c715ed02,5945,3e16ad3609f8237404cc7aeb4df7fd3a2e1f2724,2021-02-02 21:14:31,CAN,L9N0A3,9059609406,99.254.60.245,NaN,success,NaN,NaN,2021-02-02,472409******4012,2021-09,19.49


In [84]:
mask = df_charge.currency != 'USD'
tmp = df_charge[mask]
tmp_path = path_merch + "02_pure_data/tmp_ky10_for_resolve.csv"
tmp.to_csv(tmp_path, index=False, quoting=csv.QUOTE_ALL)

In [87]:
col_names = ['amount', 'amount_usd', 'currency']
tmp[col_names].head(10)

,amount,amount_usd,currency
4,25.34,19.49,CAD
5,20.29,15.61,CAD
9,25.33,19.64,AUD
11,25.36,19.51,CAD
20,508.40,391.08,CAD
28,25.31,19.47,CAD
30,46.68,35.91,CAD
33,30.37,23.36,CAD
37,30.36,23.35,CAD
39,25.36,19.51,CAD


##### Research

In [19]:
file_to_resolve_path = '/mnt/files/workdata/work/merchants/merchant_35_ky_2021-03-12/02_pure_data/ky10_for_resolve.csv'
df_orig_research =  pd.read_csv(file_to_resolve_path ,dtype=str, keep_default_na=False)

In [31]:
mask = df_orig_research.currency == 'ARS'
tmp = df_orig_research[mask]

In [51]:
col_names = ['amount', 'amount_usd', 'currency','id', 'order_id']
mask = tmp.order_id == '59291906828339593'
tmp2 = tmp[mask]
tmp2[col_names]

,amount,amount_usd,currency,id,order_id
111670,6603.44,72.73,ARS,514945******2527_2028-07,59291906828339593


In [35]:
str(list(df_orig_research))

"['id', 'order_id', 'type', 'status', 'amount', 'currency', 'terminal_id', 'mcc', 'project_id', 'date', 'country', 'zip', 'phone', 'ip', 'reported', 'status_old', 'reported_chargeback', 'date_cb_only', 'date_only', 'card_masked', 'card_expire', 'amount_usd']"

In [44]:
df_orig_research['first_card_digit'] = df_orig_research.card_masked.str[:1]

In [10]:
df_orig_research.first_card_digit.value_counts()

4    126999
5     79107
Name: first_card_digit, dtype: int64

In [15]:
mask = df_orig_research.first_card_digit == '4'
df_orig_research_4 = df_orig_research[mask]
df_orig_research_5 = df_orig_research[~mask]

In [17]:
Statistic.get_cb_rate(df_orig_research)

0.68

In [18]:
Statistic.get_cb_rate(df_orig_research_4)

0.84

In [19]:
Statistic.get_cb_rate(df_orig_research_5)

0.42

In [22]:
path_ky9 = '/mnt/files/workdata/work/merchants/merchant_32_ky9_2020-05-12_white_visa/\
02_pure_data/ky9_for_resolve.csv'

In [23]:
ky9 =  pd.read_csv(path_ky9 ,dtype=str, keep_default_na=False)

In [24]:
ky9['first_card_digit'] = ky9.card_masked.str[:1]

In [25]:
df_orig_research.first_card_digit.value_counts()

4    126999
5     79107
Name: first_card_digit, dtype: int64

In [26]:
mask = ky9.first_card_digit == '4'
ky9_4 = ky9[mask]
ky9_5 = ky9[~mask]

In [27]:
Statistic.get_cb_rate(ky9)

1.31

In [28]:
Statistic.get_cb_rate(ky9_4)

1.27

In [29]:
Statistic.get_cb_rate(ky9_5)

1.57

In [30]:
Statistic.get_stat_summarise_by_column(ky9, 'first_card_digit')

,n,amount_total,n_bad,amount_bad,cb_rate,cb_rate_amount,true_amount_weight,false_amount_weight,true_weight,false_weight,p,p_a
first_card_digit,,,,,,,,,,,,
4,3681521,3.516641e+08,46649,6.260718e+06,1.2671,1.7803,0.828240,0.867695,0.825050,0.854016,0.491374,0.488368
5,631230,5.396490e+07,9892,1.298346e+06,1.5671,2.4059,0.171761,0.132305,0.174967,0.145984,0.545152,0.564881


In [41]:
df_orig_research.amount = df_orig_research.amount_usd.copy()

In [45]:
Statistic.get_stat_summarise_by_column(df_orig_research, 'first_card_digit')

,n,amount_total,n_bad,amount_bad,cb_rate,cb_rate_amount,true_amount_weight,false_amount_weight,true_weight,false_weight,p,p_a
first_card_digit,,,,,,,,,,,,
4,126999,1.433072e+07,1064,190750.17,0.8378,1.3311,0.723841,0.620735,0.760714,0.615198,0.552880,0.538341
5,79107,8.712209e+06,335,72778.37,0.4235,0.8354,0.276195,0.379265,0.240000,0.384807,0.384119,0.421375


In [95]:
path_tmp = '/mnt/files/projects/data/result/db_teach.csv'
tmp = pd.read_csv(path_tmp, dtype=str, keep_default_na=False)

In [96]:
pd.set_option('display.max_columns', 200)

In [97]:
tmp.iloc[:,:]

,amount,amount_deviation_usd,amount_usd,area_code,bank_currency,bank_name,bin,card_type,city,client_hour,continent_code,count_months_to_end_card,country,country_code3,currency,date,date_cb_only,date_only,day_of_week,distance_haversine,distance_l2,distance_sqs_subtraction,hour,id,is_city_resolved,latitude,longitude,metro_code,order_id,phone_2_norm,region_name,status
0,141.68,3,141.68,860,840,"TD BANK, N.A.",448233,VISA card,Vernon Rockville,14,NA,14,USA,USA,USD,2021-02-02 21:10:22,undefined,2021-02-02,2,0.153902149788471,0.403481497683806,4.18798475693855,21,448233******3385_2022-04,1,41.8365,-72.4632,533,58963204904455627,60,Connecticut,0
1,26.83,0,26.83,0,840,NASSAU EDUCATORS F.C.U.,438787,VISA card,undefined,21,NA,45,USA,USA,USD,2021-02-02 21:11:20,undefined,2021-02-02,2,0.0140942632758067,0.033064662702264,7.74357209333864,21,438787******8595_2024-11,0,38.0000,-97.0000,0,58963219744750136,16,undefined,0
2,34.77,0,34.77,0,840,"BANK OF AMERICA, N.A.",481582,VISA card,undefined,21,NA,35,USA,USA,USD,2021-02-02 21:14:03,undefined,2021-02-02,2,0.0140942632758067,0.033064662702264,7.74357209333864,21,481582******0912_2024-01,0,38.0000,-97.0000,0,58963261615776513,14,undefined,0
3,25.32,-1,25.32,315,840,"STAR PROCESSING, INC.",510277,MasterCard,Utica,14,NA,42,USA,USA,USD,2021-02-02 21:14:29,undefined,2021-02-02,2,0.137687043064337,0.360984334840144,4.50621752613016,21,510277******8427_2024-08,1,43.1548,-75.1426,526,58963268351715157,15,New York,0
4,19.49,-2,19.49,0,124,TD CANADA TRUST,472409,VISA card,Toronto,14,NA,7,CAN,CAN,CAD,2021-02-02 21:14:31,undefined,2021-02-02,2,0.11229072384529,0.291189508640234,5.05343210691049,21,472409******4012_2021-09,1,43.6656,-79.3830,0,58963268900246649,05,Ontario,0
5,15.61,-2,15.61,0,124,ROYAL BANK OF CANADA,451409,VISA card,Toronto,14,NA,25,CAN,CAN,CAD,2021-02-02 21:15:43,undefined,2021-02-02,2,0.112755344681586,0.292359641933585,5.04587780337019,21,451409******5759_2023-03,1,43.7334,-79.3329,0,58963287123478142,06,Ontario,0
6,54.81,0,54.81,0,840,"CITIBANK, N.A.",542418,MasterCard,undefined,21,NA,32,USA,USA,USD,2021-02-02 21:18:43,undefined,2021-02-02,2,0.0140942632758067,0.033064662702264,7.74357209333864,21,542418******7925_2023-10,0,38.0000,-97.0000,0,58963333353970410,18,undefined,0
7,26.81,0,26.81,941,840,"WELLS FARGO BANK, N.A.",473702,VISA card,Sarasota,14,NA,42,USA,USA,USD,2021-02-02 21:23:55,undefined,2021-02-02,2,0.139580412985841,0.305768798275428,5.64040616103867,21,473702******9056_2024-08,1,27.2187,-82.4710,539,58963413215305246,41,Florida,0
8,103.41,2,103.41,0,840,"CHASE BANK USA, N.A.",426684,VISA card,undefined,21,NA,79,USA,USA,USD,2021-02-02 21:24:07,undefined,2021-02-02,2,0.0140942632758067,0.033064662702264,7.74357209333864,21,426684******9786_2027-09,0,38.0000,-97.0000,0,58963416336692708,03,undefined,0
